In [1]:
from nltk.corpus import stopwords
import nltk
import re
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences



    
    
    

In [2]:
movies = pd.read_csv("../datasets/reseñas_limpio.csv") #IMPORTO MI CSV LIMPIO

**A CONTINUACION VOY A DIVIDIR LOS DATOS IGUAL QUE HICE PARA ENTRENAR MI MODELO, VAN A TENER LAS MISMAS DIMENSIONES Y SEPARACIONES.
DE ESTAS SEPARACIONES SOLO ME VA A INTERESAR EL X ENTRENAMIENTO YA QUE CON EL ESCOGI EL VOCABULARIO DE LAS 5000 PALABRAS MÁS FRECUENTES Y FUE CON LAS PALABRAS CON LAS QUE APRENDIO, ASIQUE, CON ESAS MISMAS PALABRAS HAREMOS LAS PREDICCIONES DE LAS RESEÑAS NUEVAS QUE SE LE PRESENTEN**

In [3]:
y = movies['sentiment'] #NUESTRA VARIABLE Y CON 0 Y 1 PARA LAS ETIQUETAS DEL SENTIMIENTO

y = np.array(list(map(lambda x: 1 if x=="positive" else 0, y))) #CON ESTO LO QUE HAGO ES ESTABLECER EN 1 LOS POSITIVOS 
#Y EN 0 LOS NEGATIVOS.

X = movies["review"] #LA VARABLE X SERAN LAS LISTAS DE PALABRAS DE RESEÑAS

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
#HAGO LAS MISMAS SEPARACIONES CON EL MISMO RANDOM PARA TENER LOS MISMOS DATOS DISTRIBUIDOS, Y ASI MI MODELO FUNCIONARÁ COMO DEBE.

**AHORA CARGARE MI MODELO, QUE GUARDE EN EL OTRO CUADERNO**

In [5]:
from keras.models import model_from_json
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
 
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# se cargan los pesos (weights) en el nuevo modelo
loaded_model.load_weights("model.h5")
print("modelo cargado")


modelo cargado


In [6]:
#CREO UNA FUNCION PARA QUE ME LIMPIE TODA LA RESEÑA Y ME LA DEJE LISTA PARA PREDECIR.

In [7]:

def preprocessing_sentence(sentence):
    stoplist = set(stopwords.words("english"))
    
    
    #quito etiquetas
    sentence = re.sub(r'<[^>]*?>', '',sentence)
    #quito signos de puntuacion
    sentence = re.sub('[^a-zA-Z]', ' ',sentence)
    #quito espacios en blanco
    sentence= re.sub(r'\s+', ' ',sentence)
    #todo minusculas
    sentence = sentence.lower()
    #quitar_string_cortos
    sentence = sentence.split()
    sentence = [x for x in sentence if len(x)>=3]
    sentence = " ".join(sentence)
    #quitar palabras irrelevantes que esten en el stopwords
    sentence = sentence.split()
    sentence = [w for w in sentence if w not in stoplist]
    #convierto sentencia a serie pandas, luego a string de lista como el formato con los que entrenó el modelo
    sentence = pd.Series(str(sentence))
  
    return sentence
    
    

In [8]:

def prediccion_reseña(reseña,voc_train): #recibira la reseña procesada y el train con el que entrene el modelo
    #train = pd.read_csv("datasets/train_vocabulario.csv")
    token = Tokenizer(num_words=5000) #me quedo con las 5000 palabras mas frecuences, con las que entrenó el modelo
    token.fit_on_texts(voc_train) #actualizo vocabulario
    #convierto a numeros el entreno y la reseña
    voc_train = token.texts_to_sequences(voc_train)
    reseña = token.texts_to_sequences(reseña)
    #establezco el mismo tamaño de entrada que le meti a mi red neuronal, es decir, de una lista de 100 números.
    reseña = pad_sequences(reseña, padding='post', maxlen=100)
    #prediccion
    pred = loaded_model.predict(reseña)
    if pred > 0.5:
        print(":)")
    else:
        print(":(")
    
   
    
 
    
    

**HE COGIDO UNA RESEÑA DE UNA PAGINA WEB DE RESEÑAS DE USUARIOS DE PELICULAS, OBVIAMENTE SE NOTA QUE ES POSITIVA.**

In [9]:
r = "With its peppy cast and its brilliant, social media-hued color palette, Freaky never forgets to be anything less than a fun time."

In [10]:
reseña = preprocessing_sentence(r)
reseña #VEMOS PROCESADA LA RESEÑA.

0    ['peppy', 'cast', 'brilliant', 'social', 'medi...
dtype: object

In [11]:
prediccion_reseña(reseña,X_train) #PREDECIMOS... LA CARITA SONRIENTE INDICA QUE ES POSITIVA.

:)


**HARÉ OTRA PRUEBA CON UNA MUY MALA Y VEREMOS SI ESA CARA CAMBIA O ES LA MISMA...**

In [12]:
reseña_negativa = "the movie is a horror, they should censor it, because of course it makes me want to vomit, I'm not going to take my children to see this movie even if they give me a million euros."

In [13]:
reseña_negativa = preprocessing_sentence(reseña_negativa)
print(reseña_negativa)

0    ['movie', 'horror', 'censor', 'course', 'makes...
dtype: object


In [14]:
prediccion_reseña(reseña_negativa,X_train) #podemos observar que efectivamente, de una reseña que inventamos nosotros 
#a podido clasificarla correctamente.

:(


In [15]:
#otra reseña negativa...
otra_negativa = "Simply put, Freaky sucks."

In [16]:
otra_negativa = preprocessing_sentence(otra_negativa)
print(otra_negativa)

0    ['simply', 'put', 'freaky', 'sucks']
dtype: object


In [17]:
prediccion_reseña(otra_negativa,X_train)


:(


In [18]:
#ahora con una reseña buena

In [19]:
reseña_buena = "Sound of Metal is a film with perfect narrative rhythm; the cinematic story makes interesting, poignant decisions for the entirety of its runtime."





In [20]:
reseña_buena = preprocessing_sentence(reseña_buena)

print(reseña_buena)

0    ['sound', 'metal', 'film', 'perfect', 'narrati...
dtype: object


In [21]:
prediccion_reseña(reseña_buena,X_train) 

:)


In [22]:
#otra positiva

In [23]:
positiva = "Nomadland is a humanist masterpiece ... This is a quiet film, but never a dull one, with shots and themes that will continue to resonate "

In [24]:
positiva = preprocessing_sentence(positiva)
print(positiva)

0    ['nomadland', 'humanist', 'masterpiece', 'quie...
dtype: object


In [25]:
prediccion_reseña(positiva,X_train) 



:)


In [26]:
padrino = "One of the greatest movies of all time."

In [27]:
padrino = preprocessing_sentence(padrino)

In [28]:
prediccion_reseña(padrino,X_train) 

:)
